# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

## outline (for this course)

- OpenAi function calling
- LangChain Expression Language (LCEL)
- OpenAI function calling in LangChain
- Tagging and extraction using OpenAI function calling
- Tools and Routing
- Conversational Agent

## Function calling

- OpenAI has fine-tuned the gpt-3.5 turbo-0613 and gpt-4-0613 models to :
  1. accept additional arguments through which users can pass in descriptions of functions.
  2. if it is relevant, return the nam of the function to use, along with a JSON object with teh appropriate input parameters.


#### did not really follow the tutorials, as the syntex are basically completely different

#### base codes: https://platform.openai.com/docs/guides/function-calling

In [108]:
# !pip install python-dotenv
# !pip install openai
# !pip install -U langchain langchain-openai

In [109]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
OPENAI_API_KEY = openai.api_key

In [110]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [111]:
# define a function

# functions = [
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather in a given location",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
#             },
#             "required": ["location"],
#         },
#     }
# ]

# new syntex requires insert function into part of the tools

tools = [
    {
        "type": "function",
        "function":  {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
        }
    }
]

In [112]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [113]:
import openai

In [114]:
# an example of new syntex from: https://github.com/openai/openai-python

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key= OPENAI_API_KEY
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

In [115]:
chat_completion

ChatCompletion(id='chatcmpl-A9dVGD7BkBfTP67XXyKSXpYCk0dFx', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726860166, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [116]:
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "get_current_weather",
#             "description": "Get the current weather in a given location",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "Boston",
#                 },
#                 },
#                 "required": ["order_id"],
#                 "additionalProperties": False,
#             },
#         }
#     }
# ]

# messages = [
#     {"role": "system", "content": "You are a helpful assistant and will answer any user's questions."},
#     {"role": "user", "content": "Hi, can you tell me the weather in Boston?"}
# ]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, what is the weather like in Boston?"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # functions = functions,
    # function_call="auto",
)

In [117]:
response

ChatCompletion(id='chatcmpl-A9dVHpdlpkr7N8ZCld1BZUI58DpL2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have the capability to provide real-time weather updates. You can check a weather website or use a weather application for the most current information. If you need help with an order or delivery, feel free to ask!", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726860167, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=45, prompt_tokens=104, total_tokens=149, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [118]:
# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo-0613",
#     messages=messages,
#     functions=functions
# )
# import OpenAI

# source: https://platform.openai.com/docs/guides/chat-completions/getting-started
# https://platform.openai.com/docs/api-reference/chat/streaming

from openai import OpenAI
client = OpenAI(
    api_key= OPENAI_API_KEY
)

# response1 = client.chat.completions.create(
#   model="gpt-4o-mini",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Who won the world series in 2020?"},
#     {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
#     {"role": "user", "content": "Where was it played?"}
#   ]
# )

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages= messages,
  # new syntex
  tools=tools,
  # function_call="auto"
)



# messages = [
#     {
#         "role": "user",
#         "content": "What's the weather like in Boston?"
#     }
# ]


# from openai import OpenAI

# response = client.chat.completions.create(
#     # model="gpt-4o-mini",
#     messages=messages,
#     functions=functions,
#     model="gpt-3.5-turbo-1106",
#     # response_format={"type": "json_object"}
# )


In [119]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
# // highlight-start
messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
messages.append({"role": "user", "content": "i think it is order_12345"})
# // highlight-end

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    tools=tools
)

In [120]:
# Simulate the order_id and delivery_date
from datetime import datetime
order_id = "order_12345"
delivery_date = datetime.now()

# Simulate the tool call response
response = {
    "choices": [
        {
            "message": {
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_62136354",
                        "type": "function",
                        "function": {
                            "arguments": "{'order_id': 'order_12345'}",
                            "name": "get_delivery_date"
                        }
                    }
                ]
            }
        }
    ]
}

# Create a message containing the result of the function call
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "order_id": order_id,
        "delivery_date": delivery_date.strftime('%Y-%m-%d %H:%M:%S')
    }),
    "tool_call_id": response['choices'][0]['message']['tool_calls'][0]['id']
}

# Prepare the chat completion call payload
completion_payload = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
        {"role": "user", "content": "Hi, can you tell me the delivery date for my order?"},
        {"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"},
        {"role": "user", "content": "i think it is order_12345"},
        response['choices'][0]['message'],
        function_call_result_message
    ]
}

# Call the OpenAI API's chat completions endpoint to send the tool call result back to the model
response = openai.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)

# Print the response from the API. In this case it will typically contain a message such as "The delivery date for your order #12345 is xyz. Is there anything else I can help you with?"
print(response)

ChatCompletion(id='chatcmpl-A9dVJ4o6IXZ4wX6KQtYH4AbgnLPai', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order with ID **order_12345** is scheduled to be delivered on **September 20, 2024**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726860169, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=42, prompt_tokens=131, total_tokens=173, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))


In [121]:
response

ChatCompletion(id='chatcmpl-A9dVJ4o6IXZ4wX6KQtYH4AbgnLPai', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order with ID **order_12345** is scheduled to be delivered on **September 20, 2024**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726860169, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=42, prompt_tokens=131, total_tokens=173, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [125]:
import json

class FunctionCall:
    def __init__(self, arguments, name):
        self.arguments = arguments
        self.name = name

class ChatCompletionMessage:
    def __init__(self, content, refusal, role, function_call):
        self.content = content
        self.refusal = refusal
        self.role = role
        self.function_call = function_call

class Choice:
    def __init__(self, finish_reason, index, logprobs, message):
        self.finish_reason = finish_reason
        self.index = index
        self.logprobs = logprobs
        self.message = message

class ChatCompletion:
    def __init__(self, id, choices, created, model, object, service_tier, system_fingerprint, usage):
        self.id = id
        self.choices = choices
        self.created = created
        self.model = model
        self.object = object
        self.service_tier = service_tier
        self.system_fingerprint = system_fingerprint
        self.usage = usage

function_call = FunctionCall(arguments='{"location":"Boston, MA"}', name="get_current_weather")
message = ChatCompletionMessage(content=None, refusal=None, role="assistant", function_call=function_call)
choice = Choice(finish_reason="function_call", index=0, logprobs=None, message=message)
chat_completion = ChatCompletion(id="chatcmpl-A9cvyKJ47MV0cIw5fvFeT6ZLtfC4w", choices=[choice], created=1726857978,
                                model="gpt-3.5-turbo-1106", object="chat.completion", service_tier=None,
                                system_fingerprint="fp_e81b59fe66", usage={"completion_tokens": 17, "prompt_tokens": 82, "total_tokens": 99})

# Now manually converting the ChatCompletion object to a serializable dictionary
chat_completion_dict = {
    "id": chat_completion.id,
    "choices": [
        {
            "finish_reason": choice.finish_reason,
            "index": choice.index,
            "logprobs": choice.logprobs,
            "message": {
                "content": choice.message.content,
                "refusal": choice.message.refusal,
                "role": choice.message.role,
                "function_call": {
                    "arguments": choice.message.function_call.arguments,
                    "name": choice.message.function_call.name
                }
            }
        }
    ],
    "created": chat_completion.created,
    "model": chat_completion.model,
    "object": chat_completion.object,
    "service_tier": chat_completion.service_tier,
    "system_fingerprint": chat_completion.system_fingerprint,
    "usage": chat_completion.usage
}

# Now use json.dumps to prettify the response
pretty_response = json.dumps(chat_completion_dict, indent=4)
print(pretty_response)


{
    "id": "chatcmpl-A9cvyKJ47MV0cIw5fvFeT6ZLtfC4w",
    "choices": [
        {
            "finish_reason": "function_call",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "function_call": {
                    "arguments": "{\"location\":\"Boston, MA\"}",
                    "name": "get_current_weather"
                }
            }
        }
    ],
    "created": 1726857978,
    "model": "gpt-3.5-turbo-1106",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": "fp_e81b59fe66",
    "usage": {
        "completion_tokens": 17,
        "prompt_tokens": 82,
        "total_tokens": 99
    }
}


In [126]:
response

ChatCompletion(id='chatcmpl-A9dVJ4o6IXZ4wX6KQtYH4AbgnLPai', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order with ID **order_12345** is scheduled to be delivered on **September 20, 2024**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726860169, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=42, prompt_tokens=131, total_tokens=173, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [127]:
response_message = response.choices
response_message

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order with ID **order_12345** is scheduled to be delivered on **September 20, 2024**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))]